## PL Tables Loading Code

In [ ]:
#@title
import requests, json
from pprint import pprint

In [ ]:
#@title
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11391751}


In [ ]:
#@title
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
#@title
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,3,Xhaka,1,3
1,4,Elneny,1,3
2,5,Holding,1,2
3,6,Partey,1,3
4,7,Ødegaard,1,3


In [ ]:
#@title
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1245,1285,1250,1250,1240,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1070,1100,1070,1075,1070,1130,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,2,None,False,0,1035,1095,1020,1110,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1115,1180,1100,1160,1130,1200,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1170,1175,1140,1150,1200,1200,131


In [ ]:
#@title
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],259
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],327
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],92


In [ ]:
#@title
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Granit,Xhaka,Arsenal
1,Mohamed,Elneny,Arsenal
2,Rob,Holding,Arsenal
3,Thomas,Partey,Arsenal
4,Martin,Ødegaard,Arsenal


In [ ]:
#@title
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Granit,Xhaka,Arsenal,Midfielder
1,Mohamed,Elneny,Arsenal,Midfielder
2,Thomas,Partey,Arsenal,Midfielder
3,Martin,Ødegaard,Arsenal,Midfielder
4,Nicolas,Pépé,Arsenal,Midfielder


In [ ]:
#@title

def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
get_gameweek_history(283)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists',
        'clean_sheets',
        'saves',
        'bonus',
        'ict_index',
        'value',
        'was_home'
    ]
].head()


,round,total_points,minutes,goals_scored,assists,clean_sheets,saves,bonus,ict_index,value,was_home
0,1,12,90,1,1,0,0,2,8.5,130,False
1,2,2,90,0,0,0,0,0,14.6,130,True
2,3,8,90,1,0,0,0,1,15.6,130,False
3,4,3,90,0,0,1,0,0,8.3,130,True
4,5,10,90,0,2,0,0,2,14.7,130,True


In [ ]:
#@title
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(283)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2013/14,31,501,2,1
1,2014/15,3,33,0,0
2,2017/18,303,2905,32,12
3,2018/19,259,3254,22,12
4,2019/20,233,2879,19,10
5,2020/21,231,3077,22,6
6,2021/22,265,2758,23,14


In [ ]:
#@title
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

# rename columns
players = players.rename(
    columns={'id_player':'id_player', 'first_name':'first_name','second_name':'second_name','web_name':'web_name','name':'team','singular_name_short':'position'}
)

players.head()

,id_player,first_name,second_name,web_name,team,position
0,3,Granit,Xhaka,Xhaka,Arsenal,MID
1,4,Mohamed,Elneny,Elneny,Arsenal,MID
2,6,Thomas,Partey,Partey,Arsenal,MID
3,7,Martin,Ødegaard,Ødegaard,Arsenal,MID
4,9,Nicolas,Pépé,Pépé,Arsenal,MID


In [ ]:
#@title
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
#@title
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name','team','position']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/761 [00:00<?, ?it/s]

In [ ]:
points['goals_n_assists']=points['goals_scored']+points['assists']
points['ict_index']=pd.to_numeric(points['ict_index'])

# Enter Details

In [ ]:
#@title Enter Gameweek
curr_gw=max(points['round'])
print("Latest Gameweek is: ",curr_gw)
gw=int(input("Enter Gameweek: "))
lag=int(input("Enter #of trailing Gameweeks (0=only current gameweek, 100=all previous gameweeks this season): "))
if(gw>curr_gw):
  gw=curr_gw
if(lag>(gw-1) or lag<0):
  lag=gw-1
print("Loading stats from GW",gw-lag,"to GW",gw)

Latest Gameweek is:  33
Enter Gameweek: 33
Enter #of trailing Gameweeks (0=only current gameweek, 100=all previous gameweeks this season): 3
Loading stats from GW 30 to GW 33


#PL Stats

In [ ]:
#@title
# get top scoring players
pr_df1=points.query("round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'mean', 'assists':'sum','ict_index':'mean'}
).reset_index(
).sort_values(
    ['goals_scored','ict_index'], ascending=[False,False]
).head()

pr_df2=points.query("round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'mean', 'assists':'mean','ict_index':'mean'}
).reset_index(
).sort_values(
    ['assists','ict_index'], ascending=[False,False]
).head()

pr_df3=points.query("round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'mean', 'assists':'mean','goals_n_assists':'sum','ict_index':'mean'}
).reset_index(
).sort_values(
    ['goals_n_assists','ict_index'], ascending=[False,False]
).head()

pr_df4=points.query("round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'saves':'mean','ict_index':'mean'}
).reset_index(
).sort_values(
    ['saves','ict_index'], ascending=[False,False]
).head()

print("Gameweek",gw)
print('Most Goals')
display(pr_df1)
print('\n')
print('Most Assists')
display(pr_df2)
print('\n')
print('Most Goals+Assists')
display(pr_df3)
print('\n')
print('Most Saves')
display(pr_df4)

Gameweek 33
Most Goals


,web_name,team,position,total_points,goals_scored,assists,ict_index
287,Haaland,Man City,FWD,12.666667,1.666667,2,15.100
596,Salah,Liverpool,MID,7.000000,1.000000,0,11.500
193,Diogo Jota,Liverpool,FWD,8.000000,1.000000,1,8.375
720,Watkins,Aston Villa,FWD,6.750000,0.750000,1,9.450
223,Eze,Crystal Palace,MID,7.250000,0.750000,0,8.975




Most Assists


,web_name,team,position,total_points,goals_scored,assists,ict_index
175,De Bruyne,Man City,MID,12.000000,0.666667,1.333333,10.933333
18,Alexander-Arnold,Liverpool,DEF,6.000000,0.000000,1.250000,9.450000
423,March,Brighton,MID,5.666667,0.000000,1.000000,10.600000
503,Olise,Crystal Palace,MID,5.500000,0.000000,0.750000,8.325000
637,Solanke,Bournemouth,FWD,6.000000,0.250000,0.750000,7.750000




Most Goals+Assists


,web_name,team,position,total_points,goals_scored,assists,goals_n_assists,ict_index
287,Haaland,Man City,FWD,12.666667,1.666667,0.666667,7,15.100000
175,De Bruyne,Man City,MID,12.000000,0.666667,1.333333,6,10.933333
18,Alexander-Arnold,Liverpool,DEF,6.000000,0.000000,1.250000,5,9.450000
193,Diogo Jota,Liverpool,FWD,8.000000,1.000000,0.250000,5,8.375000
596,Salah,Liverpool,MID,7.000000,1.000000,0.000000,4,11.500000




Most Saves


,web_name,team,position,total_points,saves,ict_index
488,Neto,Bournemouth,GKP,5.000000,4.50,2.850000
335,Johnstone,Crystal Palace,GKP,5.750000,4.50,2.800000
535,Pickford,Everton,GKP,3.250000,4.50,2.550000
39,Arrizabalaga,Chelsea,GKP,3.333333,4.00,3.333333
556,Raya,Brentford,GKP,4.000000,3.75,2.875000


# Best Players

In [ ]:
#@title
pr_df1=points.query("position=='FWD' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'sum', 'assists':'sum','ict_index':'sum'}
).reset_index(
).sort_values(
    ['total_points','ict_index'], ascending=[False,False]
).head()

pr_df2=points.query("position=='MID' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'sum', 'assists':'sum','ict_index':'sum'}
).reset_index(
).sort_values(
    ['total_points','ict_index'], ascending=[False,False]
).head()

pr_df3=points.query("position=='DEF' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'sum', 'assists':'sum','ict_index':'sum'}
).reset_index(
).sort_values(
    ['total_points','ict_index'], ascending=[False,False]
).head()

pr_df4=points.query("position=='GKP' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'mean', 'goals_scored':'sum', 'assists':'sum','ict_index':'sum'}
).reset_index(
).sort_values(
    ['total_points','ict_index'], ascending=[False,False]
).head()

print("Top Strikers")
display(pr_df1)
print('\n')
print("Top Midfielders")
display(pr_df2)
print('\n')
print("Top Defenders")
display(pr_df3)
print('\n')
print("Top Goal-Keepers")
display(pr_df4)


Top Strikers


,web_name,team,position,total_points,goals_scored,assists,ict_index
38,Haaland,Man City,FWD,12.666667,5,2,45.3
24,Diogo Jota,Liverpool,FWD,8.000000,4,1,33.5
48,Kane,Spurs,FWD,7.000000,2,2,24.2
85,Watkins,Aston Villa,FWD,6.750000,3,1,37.8
78,Solanke,Bournemouth,FWD,6.000000,1,3,31.0




Top Midfielders


,web_name,team,position,total_points,goals_scored,assists,ict_index
71,De Bruyne,Man City,MID,12.00,2,4,32.8
145,Joelinton,Newcastle,MID,7.50,2,2,15.2
92,Eze,Crystal Palace,MID,7.25,3,0,35.9
270,Salah,Liverpool,MID,7.00,4,0,46.0
186,Martinelli,Arsenal,MID,7.00,2,2,43.4




Top Defenders


,web_name,team,position,total_points,goals_scored,assists,ict_index
155,Mings,Aston Villa,DEF,7.250000,1,0,14.6
4,Alexander-Arnold,Liverpool,DEF,6.000000,0,5,37.8
258,Álex Moreno,Aston Villa,DEF,6.000000,0,1,18.3
207,Stones,Man City,DEF,5.666667,2,0,11.5
95,Guéhi,Crystal Palace,DEF,5.500000,1,0,18.3




Top Goal-Keepers


,web_name,team,position,total_points,goals_scored,assists,ict_index
35,Johnstone,Crystal Palace,GKP,5.750000,0,0,11.2
73,Sá,Wolves,GKP,5.500000,0,0,9.0
45,Martínez,Aston Villa,GKP,5.250000,0,0,5.7
52,Neto,Bournemouth,GKP,5.000000,0,0,11.4
17,De Gea,Man Utd,GKP,4.666667,0,0,1.4


# Best Gameweek Teams

In [ ]:
#@title
pr_df1=points.query("(position=='MID' or position=='FWD') and round<=@gw and round>=(@gw-@lag)"
).groupby(
    ['team']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'goals_conceded':'max'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(10)

pr_df2=points.query("(position=='DEF' or position=='GKP') and round<=@gw and round>=(@gw-@lag)"
).groupby(
    ['team']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'goals_conceded':'max'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(10)

#@title
pr_df3=points.query("(position=='MID' or position=='FWD') and round<=@gw and round>=(@gw-@lag)"
).groupby(
    ['team']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'goals_conceded':'max'}
).reset_index(
).sort_values(
    'total_points', ascending=True
).head(10)

pr_df4=points.query("(position=='DEF' or position=='GKP') and round<=@gw and round>=(@gw-@lag)"
).groupby(
    ['team']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'goals_conceded':'max'}
).reset_index(
).sort_values(
    'total_points', ascending=True
).head(10)

print("Top Attack")
display(pr_df1)
print('\n')
print("Top Defence")
display(pr_df2)
print('\n')
print("Worst Attack")
display(pr_df3)
print('\n')
print("Worst Defence")
display(pr_df4)

Top Attack


,team,total_points,goals_scored,goals_conceded
14,Newcastle,146,11,3
11,Liverpool,139,12,2
12,Man City,128,9,1
6,Crystal Palace,125,6,2
18,West Ham,117,7,2
1,Aston Villa,115,6,1
0,Arsenal,114,7,4
19,Wolves,109,5,2
13,Man Utd,102,5,2
2,Bournemouth,99,4,4




Top Defence


,team,total_points,goals_scored,goals_conceded
1,Aston Villa,121,1,1
19,Wolves,107,0,2
18,West Ham,91,0,2
6,Crystal Palace,89,1,2
2,Bournemouth,84,1,4
13,Man Utd,78,1,2
3,Brentford,74,0,2
11,Liverpool,63,1,2
14,Newcastle,59,0,3
10,Leicester,49,1,3




Worst Attack


,team,total_points,goals_scored,goals_conceded
5,Chelsea,47,1,2
7,Everton,72,2,4
15,Nott'm Forest,72,3,3
4,Brighton,73,3,3
10,Leicester,77,3,3
9,Leeds,79,3,6
16,Southampton,82,3,4
3,Brentford,83,3,2
17,Spurs,94,6,6
8,Fulham,97,5,1




Worst Defence


,team,total_points,goals_scored,goals_conceded
9,Leeds,7,0,6
5,Chelsea,24,0,2
16,Southampton,26,1,4
0,Arsenal,31,1,4
4,Brighton,36,1,3
17,Spurs,36,1,6
15,Nott'm Forest,41,1,3
8,Fulham,43,0,1
12,Man City,45,2,1
7,Everton,46,0,4


# ICT Stats

In [ ]:
#@title
pr_df1=points.query("(position=='MID' or position=='FWD') and round<=@gw and round>=(@gw-@lag)"
).groupby(
    ['team']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'goals_conceded':'max','ict_index':'sum'}
).reset_index(
).sort_values(
    'ict_index', ascending=False
).head(10)

pr_df2=points.query("(position=='DEF' or position=='GKP') and round<=@gw and round>=(@gw-@lag)"
).groupby(
    ['team']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'goals_conceded':'max','ict_index':'sum'}
).reset_index(
).sort_values(
    'ict_index', ascending=False
).head(10)

print("Top Attack ICT")
display(pr_df1)
print('\n')
print("Top Defence ICT")
display(pr_df2)

Top Attack ICT


,team,total_points,goals_scored,goals_conceded,ict_index
11,Liverpool,139,12,2,196.7
0,Arsenal,114,7,4,157.9
4,Brighton,73,3,3,152.2
12,Man City,128,9,1,148.7
6,Crystal Palace,125,6,2,138.8
18,West Ham,117,7,2,137.5
1,Aston Villa,115,6,1,130.7
13,Man Utd,102,5,2,126.9
9,Leeds,79,3,6,121.1
19,Wolves,109,5,2,119.6




Top Defence ICT


,team,total_points,goals_scored,goals_conceded,ict_index
11,Liverpool,63,1,2,111.9
10,Leicester,49,1,3,77.6
18,West Ham,91,0,2,72.5
8,Fulham,43,0,1,71.1
0,Arsenal,31,1,4,67.7
14,Newcastle,59,0,3,64.3
6,Crystal Palace,89,1,2,63.7
15,Nott'm Forest,41,1,3,63.7
2,Bournemouth,84,1,4,62.2
17,Spurs,36,1,6,61.6


In [ ]:
#@title
pr_df1=points.query("position=='FWD' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum','ict_index':'mean'}
).reset_index(
).sort_values(
    ['ict_index'], ascending=False
).head()

pr_df2=points.query("position=='MID' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum','ict_index':'mean'}
).reset_index(
).sort_values(
    ['ict_index'], ascending=False
).head()

pr_df3=points.query("position=='DEF' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum','ict_index':'mean'}
).reset_index(
).sort_values(
    ['ict_index'], ascending=False
).head()

pr_df4=points.query("position=='GKP' and round<=@gw and round>=(@gw-@lag)").groupby(
    ['web_name','team','position']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum','ict_index':'mean'}
).reset_index(
).sort_values(
    ['ict_index'], ascending=False
).head()

print("Top ICT Strikers")
display(pr_df1)
print('\n')
print("Top ICT Midfielders")
display(pr_df2)
print('\n')
print("Top ICT Defenders")
display(pr_df3)
print('\n')
print("Top ICT Goal-Keepers")
display(pr_df4)


Top ICT Strikers


,web_name,team,position,total_points,goals_scored,assists,ict_index
38,Haaland,Man City,FWD,38,5,2,15.100
85,Watkins,Aston Villa,FWD,27,3,1,9.450
24,Diogo Jota,Liverpool,FWD,32,4,1,8.375
81,Toney,Brentford,FWD,17,2,0,8.050
78,Solanke,Bournemouth,FWD,24,1,3,7.750




Top ICT Midfielders


,web_name,team,position,total_points,goals_scored,assists,ict_index
270,Salah,Liverpool,MID,28,4,0,11.500000
71,De Bruyne,Man City,MID,36,2,4,10.933333
186,Martinelli,Arsenal,MID,28,2,2,10.850000
183,March,Brighton,MID,17,0,3,10.600000
94,Fernandes,Man Utd,MID,11,0,1,10.366667




Top ICT Defenders


,web_name,team,position,total_points,goals_scored,assists,ict_index
4,Alexander-Arnold,Liverpool,DEF,24,0,5,9.450000
186,Robertson,Liverpool,DEF,12,0,2,6.550000
77,Dunk,Brighton,DEF,11,1,0,5.633333
227,Trippier,Newcastle,DEF,7,0,0,5.400000
170,Perišić,Spurs,DEF,7,0,2,5.400000




Top ICT Goal-Keepers


,web_name,team,position,total_points,goals_scored,assists,ict_index
4,Arrizabalaga,Chelsea,GKP,10,0,0,3.333333
60,Raya,Brentford,GKP,16,0,0,2.875000
52,Neto,Bournemouth,GKP,20,0,0,2.850000
35,Johnstone,Crystal Palace,GKP,23,0,0,2.800000
58,Ramsdale,Arsenal,GKP,5,0,0,2.775000


# My FPL Team Stats

In [ ]:
#@title
import os
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

my_team_url = 'https://fantasy.premierleague.com/api/entry/426536/event/' + \
    str(gw) + '/picks/'
my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)
my_team = my_team.merge(
    points,
    left_on='element',
    right_on='id_player',
    suffixes=['_playerx', None]
).drop(['element_playerx', 'position_playerx'], axis=1)

In [ ]:
#@title
# join team name

my_team.query("round==@gw"
).groupby(
    ['web_name','position','value','team','multiplier','round']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum', 'goals_n_assists':'sum','ict_index':'sum'}
).reset_index(
).sort_values(
    ['total_points','ict_index'], ascending=[False,False]
).head(20)


,web_name,position,value,team,multiplier,round,total_points,goals_scored,assists,goals_n_assists,ict_index
3,Haaland,FWD,123,Man City,2,33,14,1,2,3,21.6
14,Wilson,FWD,69,Newcastle,1,33,13,2,0,2,0.0
6,Raya,GKP,49,Brentford,1,33,9,0,0,0,3.5
2,Gibbs-White,MID,55,Nott'm Forest,0,33,7,1,0,1,10.2
5,Mee,DEF,51,Brentford,1,33,6,0,0,0,2.2
4,March,MID,52,Brighton,1,33,5,0,1,1,11.2
1,Eze,MID,56,Crystal Palace,1,33,2,0,0,0,9.8
8,Salah,MID,130,Liverpool,1,33,2,0,0,0,5.1
12,Watkins,FWD,77,Aston Villa,1,33,2,0,0,0,2.7
7,Saka,MID,84,Arsenal,1,33,2,0,0,0,0.9


In [ ]:
for i in range(1,gw+1):
  my_team2_url2 = 'https://fantasy.premierleague.com/api/entry/426536/event/' + \
    str(i) + '/picks/'
  my_team2 = get(my_team2_url2)['picks']
  my_team2 = pd.DataFrame(my_team2)
  my_team2 = my_team2.merge(
    points,
    left_on='element',
    right_on='id_player',
    suffixes=['_playerx', None]).drop(['element_playerx', 'position_playerx'], axis=1)
  print('Gameweek ',i,' Points:', sum(my_team2.loc[my_team2['round']==i]['total_points']*my_team2.loc[my_team2['round']==i]['multiplier'])) 

Gameweek  1  Points: 51
Gameweek  2  Points: 61
Gameweek  3  Points: 52
Gameweek  4  Points: 76
Gameweek  5  Points: 81
Gameweek  6  Points: 42
Gameweek  7  Points: 0
Gameweek  8  Points: 72
Gameweek  9  Points: 89
Gameweek  10  Points: 72
Gameweek  11  Points: 59
Gameweek  12  Points: 56
Gameweek  13  Points: 48
Gameweek  14  Points: 71
Gameweek  15  Points: 56
Gameweek  16  Points: 38
Gameweek  17  Points: 91
Gameweek  18  Points: 51
Gameweek  19  Points: 79
Gameweek  20  Points: 74
Gameweek  21  Points: 78
Gameweek  22  Points: 88
Gameweek  23  Points: 75
Gameweek  24  Points: 49
Gameweek  25  Points: 90
Gameweek  26  Points: 40
Gameweek  27  Points: 100
Gameweek  28  Points: 57
Gameweek  29  Points: 113
Gameweek  30  Points: 54
Gameweek  31  Points: 54
Gameweek  32  Points: 60
Gameweek  33  Points: 72


# Main Table Sample

In [ ]:
points.head()

,id_player,web_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,goals_n_assists
0,3,Xhaka,Arsenal,MID,3,1,7,2,False,2022-08-05T19:00:00Z,0,2,1,90,0,0,1,0,0,0,0,1,0,0,0,12,16.6,15.0,2.0,3.4,1,0.00,0.06,0.06,1.21,50,0,48303,0,0,0
1,3,Xhaka,Arsenal,MID,3,11,10,12,True,2022-08-13T14:00:00Z,4,2,2,90,1,1,0,2,0,0,0,0,0,0,2,35,54.8,25.5,28.0,10.8,1,0.38,0.10,0.48,0.46,50,-629,65418,9001,9630,2
2,3,Xhaka,Arsenal,MID,3,21,3,6,False,2022-08-20T16:30:00Z,0,3,3,87,0,1,1,0,0,0,0,0,0,0,0,25,25.8,34.0,6.0,6.6,1,0.00,0.11,0.11,0.26,50,112040,216726,137326,25286,1
3,3,Xhaka,Arsenal,MID,3,31,9,2,True,2022-08-27T16:30:00Z,2,1,4,90,0,0,0,1,0,0,0,0,0,0,0,9,8.2,25.0,12.0,4.5,1,0.09,0.07,0.16,0.83,50,42760,267951,77459,34699,0
4,3,Xhaka,Arsenal,MID,3,41,2,2,True,2022-08-31T18:30:00Z,2,1,5,90,0,0,0,1,0,0,0,0,0,0,0,14,12.6,25.0,8.0,4.6,1,0.00,0.04,0.04,0.45,50,10781,288460,49435,38654,0


In [ ]:

# Make sure to keep the trailing “/” in the url!
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
response = json.loads(response.content)

In [ ]:
players = response['elements']
teams = response['teams']
events = response['events']
players_df = pd.DataFrame(players)
teams_df = pd.DataFrame(teams)
events_df = pd.DataFrame(events)

In [ ]:
events_df['deadline_time'] = pd.to_datetime(events_df['deadline_time'])
events_df['deadline_time'] = events_df['deadline_time'].dt.tz_localize(None)
events_df[gw-5:gw]

,id,name,deadline_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,highest_score,is_previous,is_current,is_next,cup_leagues_created,h2h_ko_matches_created,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
28,29,Gameweek 29,2023-04-01 10:00:00,80,True,True,7385192.0,1680343200,0,173.0,False,False,False,True,True,"[{'chip_name': 'bboost', 'num_played': 1098572...",318.0,333.0,356.0,"{'id': 356, 'points': 19}",10235681,318.0,318.0
29,30,Gameweek 30,2023-04-08 10:00:00,54,True,True,602329.0,1680948000,0,112.0,False,False,False,True,True,"[{'chip_name': 'bboost', 'num_played': 46175},...",318.0,318.0,159.0,"{'id': 159, 'points': 14}",6552257,318.0,335.0
30,31,Gameweek 31,2023-04-15 10:00:00,55,True,True,6097158.0,1681552800,0,132.0,False,False,False,True,True,"[{'chip_name': 'bboost', 'num_played': 38298},...",318.0,305.0,169.0,"{'id': 169, 'points': 16}",7432142,318.0,427.0
31,32,Gameweek 32,2023-04-21 17:30:00,42,True,True,11367172.0,1682098200,0,115.0,True,False,False,True,True,"[{'chip_name': 'bboost', 'num_played': 9705}, ...",318.0,169.0,365.0,"{'id': 365, 'points': 13}",13224881,318.0,427.0
32,33,Gameweek 33,2023-04-25 17:00:00,55,True,True,10758293.0,1682442000,0,138.0,False,True,False,True,True,"[{'chip_name': 'bboost', 'num_played': 47883},...",318.0,318.0,301.0,"{'id': 301, 'points': 19}",6360867,318.0,283.0
